# Feature Engineering Lab: Credit Risk Signals

## Objective
Clean, engineer, and explain features for a dataset with: `daily_revenue`, `daily_expenses`, `ad_spend`, `ad_conversions`, `transaction_count`.
Goal: Predict `loan_repaid`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: '%.2f' % x)

## 1. Data Generation (Simulating the User's Schema)
We treat this as **Panel Data**: Daily observations per business.

In [ ]:
def generate_daily_data(n_businesses=50, days=90):
    np.random.seed(42)
    data = []
    
    for bid in range(n_businesses):
        business_id = f"SME-{bid:03d}"
        
        # Determine 'True' risk profile
        is_risky = np.random.random() < 0.3
        base_revenue = np.random.uniform(5000, 20000) if not is_risky else np.random.uniform(2000, 8000)
        volatility = 0.2 if not is_risky else 0.8  # Risky businesses are volatile
        growth = 0.005 if not is_risky else -0.002 # Risky businesses decline
        
        loan_repaid = 0 if is_risky else 1
        
        for day in range(days):
            trend_factor = (1 + growth) ** day
            noise = np.random.normal(1, volatility)
            
            daily_rev = max(0, base_revenue * trend_factor * noise)
            daily_exp = max(0, daily_rev * np.random.uniform(0.6, 1.1)) # Expenses track revenue but fluctuate
            
            # Missing Data Simulation (Cleanliness)
            if np.random.random() < 0.05:
                daily_rev = np.nan
                
            # Ad Spend
            ad_spend = 0
            ad_conv = 0
            if day % 7 == 0: # Weekly campaigns
                 ad_spend = np.random.uniform(1000, 5000)
                 ad_conv = int(ad_spend / np.random.uniform(500, 2000)) if not is_risky else int(ad_spend / 5000)
            
            txn_count = int(daily_rev / 500) if not pd.isna(daily_rev) else 0
            
            data.append({
                'business_id': business_id,
                'day': day,
                'daily_revenue': daily_rev,
                'daily_expenses': daily_exp,
                'ad_spend': ad_spend,
                'ad_conversions': ad_conv,
                'transaction_count': txn_count,
                'loan_repaid': loan_repaid
            })
            
    return pd.DataFrame(data)

df_raw = generate_daily_data()
print(f"Dataset Shape: {df_raw.shape}")
display(df_raw.head())

## 2. Data Cleaning
**Problem**: `daily_revenue` has missing values (simulated 5%).
**Strategy**: Forward Fill (ffill) assuming business continuity, then fill remaining with 0.

In [ ]:
# Check missing
print("Missing before clean:\n", df_raw.isnull().sum())

# Impute
# grouping by business_id is critical so we don't leak data across businesses
df_clean = df_raw.copy()
df_clean['daily_revenue'] = df_clean.groupby('business_id')['daily_revenue'].ffill().fillna(0)

# Outlier Handling (Cap at 99th percentile to remove extreme spikes)
cap_rev = df_clean['daily_revenue'].quantile(0.99)
df_clean['daily_revenue'] = df_clean['daily_revenue'].clip(upper=cap_rev)

print("\nMissing after clean:\n", df_clean.isnull().sum())

## 3. Feature Engineering
We need to transform this **Time Series** data into a **Cross-Sectional** dataset (one row per business) for the model.

### Feature Group 1: Cash Flow Stability
**Intuition**: Lenders love stability. Volatile revenue suggests risk.

In [ ]:
def engineer_stability_features(df):
    # Coefficient of Variation (CV) = StdDev / Mean
    # Lower is better (more stable)
    stability = df.groupby('business_id')['daily_revenue'].agg(['std', 'mean'])
    stability['revenue_cv'] = stability['std'] / (stability['mean'] + 1) # +1 avoids div by zero
    return stability[['revenue_cv']]

### Feature Group 2: Profitability & Burn Rate
**Intuition**: Does the business make money? How fast do they spend it?

In [ ]:
def engineer_profitability_features(df):
    group = df.groupby('business_id')
    
    # Total sums
    totals = group[['daily_revenue', 'daily_expenses']].sum()
    
    # Net Income
    totals['net_income'] = totals['daily_revenue'] - totals['daily_expenses']
    
    # Profit Margin
    totals['profit_margin'] = totals['net_income'] / (totals['daily_revenue'] + 1)
    
    # Burn Rate (Expenses / Revenue)
    totals['burn_rate'] = totals['daily_expenses'] / (totals['daily_revenue'] + 1)
    
    return totals[['net_income', 'profit_margin', 'burn_rate']]

### Feature Group 3: Growth Trends
**Intuition**: Is the business growing or dying? We fit a simple linear texture to the revenue curve.

In [ ]:
from scipy.stats import linregress

def calculate_trend(series):
    # Slope of revenue over time
    slope, _, _, _, _ = linregress(range(len(series)), series)
    return slope

def engineer_growth_features(df):
    # Apply trend calculation per business
    trends = df.groupby('business_id')['daily_revenue'].apply(calculate_trend).reset_index()
    trends.columns = ['business_id', 'revenue_trend_slope']
    return trends.set_index('business_id')

### Feature Group 4: Ad Efficiency (Alternative Data)
**Intuition**: Smart businesses get high ROI. High spend with low conversions = waste/desperation.

In [ ]:
def engineer_ad_features(df):
    group = df.groupby('business_id')[['ad_spend', 'ad_conversions']].sum()
    
    # Cost Per Conversion (Lower is better)
    group['ad_cpa'] = group['ad_spend'] / (group['ad_conversions'] + 1)
    
    return group[['ad_cpa']]

## 4. Aggregation and Modeling Prep
Combine all features into the final training set.

In [ ]:
# 1. Compute features
feat_stability = engineer_stability_features(df_clean)
feat_profit = engineer_profitability_features(df_clean)
feat_growth = engineer_growth_features(df_clean)
feat_ads = engineer_ad_features(df_clean)

# 2. Get Target (loan_repaid)
# It's constant per business, so just take max or mean
target = df_clean.groupby('business_id')['loan_repaid'].max()

# 3. Merge all
df_final = pd.concat([feat_stability, feat_profit, feat_growth, feat_ads, target], axis=1)

print("Final Modeling Dataset:")
display(df_final.head())

# Correlation Check with Target
print("\nFeature Correlations with Repayment:\n")
print(df_final.corr()['loan_repaid'].sort_values(ascending=False))

## Summary of Insights
- **revenue_trend_slope**: Positive slope should strongly correlate with Repayment.
- **revenue_cv**: High volatility (high CV) should negatively correlate.
- **profit_margin**: Higher margin -> Higher repayment ability.
- **ad_cpa**: Lower cost per acquisition -> Better business efficiency.